In [64]:
## h2h
import requests
import numpy as np
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

def odds_conv(odds):
    """
    Convert odds to implied probabilities.
    """
    if odds > 0:
        return 1 / (odds / 100 + 1)
    else:
        return 1 / (1 + 100 / abs(odds))

def fair_odds(odds_a, odds_b):
    """
    Calculate fair odds for two events.
    """
    fair_a = odds_conv(odds_a) / (odds_conv(odds_a) + odds_conv(odds_b))
    fair_b = odds_conv(odds_b) / (odds_conv(odds_a) + odds_conv(odds_b))
    return (fair_a, fair_b)

def avg(x, y):
    """
    Calculate the average of two numbers.
    """
    return (x + y) / 2

def profit(stake, odds):
    """
    Calculate profit from a bet.
    """
    if odds > 0:
        return odds / 100 * stake
    else:
        return 100 / abs(odds) * stake

def kelly_criterion(b, p, q):
    """
    Calculate the Kelly Criterion for optimal betting.
    """
    return (b * p - q) / b

def width(a, b):
    """
    Calculate the width between two numbers.
    """
    if a < 0 and b < 0:
        return abs(int(str(a)[-2:]) + int(str(b)[-2:]))
    else:
        return abs(int(str(a)[-2:]) - int(str(b)[-2:]))

def get_sport_odds(api_key, sport_key='upcoming', regions='us', market='h2h', oddsFormat='american'):
    """
    Fetches sports odds data from the Odds API.
    """
    base_url = f'https://api.the-odds-api.com/v4/sports/{sport_key}/odds'
    params = {
        'apiKey': api_key,
        'regions': regions,
        'oddsFormat': oddsFormat,
        'markets': market,
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f'Failed to retrieve data. Status code: {response.status_code}')
        return None

def process_odds_data(data):
    """
    Processes the odds data and creates a DataFrame.
    """
    data_list = []
    for game in data:
        game_date = game['commence_time']
        sport = game['sport_title']
        home_team = game['home_team']
        away_team = game['away_team']
        for bookmaker in game['bookmakers']:
            bookmaker_name = bookmaker['title']
            for market in bookmaker['markets']:
                if market['key'] == 'h2h':
                    for outcome in market['outcomes']:
                        team_name = outcome['name']
                        if team_name.lower() == home_team.lower():
                            team_type = 'home'
                        elif team_name.lower() == away_team.lower():
                            team_type = 'away'
                        else:
                            team_type = 'unknown'
                        row = {
                            'date': game_date,
                            'sport': sport,
                            'home_team': home_team,
                            'away_team': away_team,
                            'bookmaker': bookmaker_name,
                            'team_type': team_type,
                            'price': outcome['price']
                        }
                        data_list.append(row)
    return pd.DataFrame(data_list)

def pivot_and_clean_data(df):
    """
    Pivots the DataFrame and fills NaN values with a placeholder.
    """
    pivoted_df = df.pivot_table(index=['date', 'sport', 'home_team', 'away_team', 'team_type'],
                                columns='bookmaker',
                                values='price',
                                aggfunc='first')
    pivoted_df.reset_index(inplace=True)
    pivoted_df['team_type'] = pivoted_df.apply(lambda row: row['away_team'] if row['team_type'] == 'away' else row['home_team'], axis=1)
    pivoted_df.rename(columns={'team_type': 'team'}, inplace=True)
    pivoted_df.fillna('N/A', inplace=True)
    return pivoted_df

def calculate_ev(dk, pin, bmgm, stake, bankroll):
    """
    Calculate expected value and optimal bet.
    """
    fair_pin = fair_odds(pin[0], pin[1])
    fair_bmgm = fair_odds(bmgm[0], bmgm[1])
    fair = (avg(fair_pin[0], fair_bmgm[0]), avg(fair_pin[1], fair_bmgm[1]))

    a_profit = profit(stake, dk[0])
    b_profit = profit(stake, dk[1])
    a_ev = a_profit * fair[0] - stake * fair[1]
    b_ev = b_profit * fair[1] - stake * fair[0]

    dkfair = fair_odds(dk[0], dk[1])

    if a_ev > b_ev:
        bet = bankroll * kelly_criterion(1, dkfair[0], dkfair[1])
        width_a = width(pin[0], pin[1])
        return 0, a_ev, dk[0], bet, width_a
    else:
        bet = bankroll * kelly_criterion(1, dkfair[0], dkfair[1])
        width_b = width(pin[0], pin[1])
        return 1, b_ev, dk[1], bet, width_b

SPORTBOOK = 'BetMGM'
def main(api_key, sport_key, regions, market, odds_format, stake, bankroll):
    """
    Main function to fetch, process, and analyze sports odds data.
    """
    data = get_sport_odds(api_key, sport_key, market, regions, odds_format)
    if data:
        processed_data = process_odds_data(data)
        pivoted_data = pivot_and_clean_data(processed_data)

        test = pivoted_data[['date', 'sport', 'home_team', 'away_team', 'team', SPORTBOOK, 'BetOnline.ag', 'Bovada']]
        test['matchup'] = test['home_team'] + ' vs ' + test['away_team']
        test['team_x'] = np.where(test['home_team'] == test['team'], 'home', 'away')
        test = test.sort_values(['matchup', 'team_x'], ascending=False)

        test['check'] = np.where((test[SPORTBOOK] == 'N/A') | (test['Bovada'] == 'N/A') | (test['BetOnline.ag'] == 'N/A'), 1, 0)
        test = test[test['check'] == 0]
        
        cols = [SPORTBOOK, 'BetOnline.ag', 'Bovada']
        for col in cols:
            test[col] = test[col].astype('int')

        consolidated_df = test.groupby(['date', 'sport', 'home_team', 'away_team']).agg({
            SPORTBOOK: tuple,
            'BetOnline.ag': tuple,
            'Bovada': tuple
        }).reset_index()

        result_df = consolidated_df.apply(lambda row: calculate_ev(row[SPORTBOOK], row['BetOnline.ag'], row['Bovada'], stake, bankroll), axis=1, result_type='expand')

        result_df.columns = ['team_index', 'ev', 'odds', 'bet', 'width']
        final_df = pd.concat([consolidated_df, result_df], axis=1)

        final_df['team'] = np.where(final_df['team_index'] == 0, final_df['home_team'], final_df['away_team'])
        final_df = final_df.sort_values('ev', ascending=False)

        return final_df

# Example usage in the main() function
if __name__ == "__main__":
    API_KEY = '287f26ecf5b58ab73781cfaf78310c08'
    SPORT_KEY = 'americanfootball_ncaaf'
    MARKET = 'h2h'
    REGIONS = 'us'
    ODDS_FORMAT = 'american'
    STAKE = 100
    BANKROLL = 1500
    final_dataframe = main(API_KEY, SPORT_KEY, MARKET, REGIONS, ODDS_FORMAT, STAKE, BANKROLL)
final_dataframe.head(20)


,date,sport,home_team,away_team,BetMGM,BetOnline.ag,Bovada,team_index,ev,odds,bet,width,team
6,2023-10-14T00:00:00Z,NCAAF,Utah State Aggies,Fresno State Bulldogs,"(210, -275)","(200, -238)","(180, -220)",0.0,2.791833,210.0,-583.503055,38.0,Utah State Aggies
2,2023-10-12T01:00:00Z,NCAAF,New Mexico State Aggies,Sam Houston State Bearkats,"(-200, 165)","(-179, 160)","(-170, 145)",1.0,1.772964,165.0,415.662651,19.0,Sam Houston State Bearkats
9,2023-10-14T16:00:00Z,NCAAF,Eastern Michigan Eagles,Kent State Golden Flashes,"(-450, 325)","(-370, 305)","(-400, 300)",1.0,1.331840,325.0,829.949239,65.0,Kent State Golden Flashes
31,2023-10-14T23:30:00Z,NCAAF,North Carolina Tar Heels,Miami Hurricanes,"(-200, 165)","(-175, 155)","(-180, 155)",1.0,0.722908,165.0,415.662651,20.0,Miami Hurricanes
23,2023-10-14T19:30:00Z,NCAAF,Western Michigan Broncos,Miami (OH) RedHawks,"(275, -350)","(265, -312)","(255, -310)",0.0,0.706744,275.0,-734.042553,53.0,Western Michigan Broncos
3,2023-10-12T23:00:00Z,NCAAF,Houston Cougars,West Virginia Mountaineers,"(125, -150)","(115, -135)","(115, -135)",0.0,0.666349,125.0,-223.404255,20.0,Houston Cougars
15,2023-10-14T19:30:00Z,NCAAF,Army Black Knights,Troy Trojans,"(150, -185)","(170, -196)","(175, -210)",1.0,-0.480827,-185.0,-356.187291,26.0,Troy Trojans
18,2023-10-14T19:30:00Z,NCAAF,Maryland Terrapins,Illinois Fighting Illini,"(-650, 450)","(-556, 430)","(-625, 430)",1.0,-0.547029,450.0,979.768786,26.0,Illinois Fighting Illini
29,2023-10-14T23:00:00Z,NCAAF,Washington State Cougars,Arizona Wildcats,"(-350, 260)","(-303, 250)","(-300, 250)",1.0,-0.778719,260.0,710.526316,47.0,Arizona Wildcats
12,2023-10-14T16:00:00Z,NCAAF,Rutgers Scarlet Knights,Michigan State Spartans,"(-225, 180)","(-204, 175)","(-200, 170)",1.0,-0.797688,180.0,479.057592,71.0,Michigan State Spartans


In [65]:
import requests
import pandas as pd

def spreads(api_key='287f26ecf5b58ab73781cfaf78310c08', regions='us', odds_format='american', markets='spreads'):
    base_url = f'https://api.the-odds-api.com/v4/sports/upcoming/odds'
    params = {
        'apiKey': api_key,
        'regions': regions,
        'oddsFormat': odds_format,
        'markets': markets
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data_list = []

        for game in response.json():
            game_date = game['commence_time']
            sport_key = game['sport_key']
            home_team = game['home_team']
            away_team = game['away_team']
            for bookmaker in game['bookmakers']:
                bookmaker_name = bookmaker['title']
                for market in bookmaker['markets']:
                    if market['key'] == 'spreads':
                        for outcome in market.get('outcomes', []):
                            team_name = outcome['name']
                            spread = outcome.get('point', 'N/A')
                            price = outcome.get('price', 'N/A')
                            if team_name.lower() == home_team.lower():
                                team_type = 'home'
                            elif team_name.lower() == away_team.lower():
                                team_type = 'away'
                            else:
                                team_type = 'unknown'

                            data_list.append({
                                'date': game_date,
                                'sport': sport_key,
                                'home_team': home_team,
                                'away_team': away_team,
                                'team_type': team_type,
                                'bookmaker': bookmaker_name,
                                'spread': spread,
                                'price': price
                            })

        df = pd.DataFrame(data_list)
        return df

    else:
        print(f'Failed to retrieve data. Status code: {response.status_code}')

def pivot_and_clean_data(df):
    """
    Pivots the DataFrame and fills NaN values with a placeholder.
    """
    # Pivot the DataFrame
    pivoted_df = df.pivot_table(index=['date', 'sport', 'home_team', 'away_team', 'team_type','spread'],
                                columns='bookmaker',
                                values='price',
                                aggfunc='first')

    pivoted_df.reset_index(inplace=True)
    pivoted_df['team_type'] = pivoted_df.apply(lambda row: row['away_team'] if row['team_type'] == 'away' else row['home_team'], axis=1)
    pivoted_df.rename(columns={'team_type': 'team'}, inplace=True)
    pivoted_df.fillna('N/A', inplace=True)

    return pivoted_df


def calculate_ev(dk, pin, bmgm, stake, bankroll):
    """
    Calculate expected value and optimal bet.
    """
    fair_pin = fair_odds(pin[0], pin[1])
    fair_bmgm = fair_odds(bmgm[0], bmgm[1])
    fair = (avg(fair_pin[0], fair_bmgm[0]), avg(fair_pin[1], fair_bmgm[1]))

    a_profit = profit(stake, dk[0])
    b_profit = profit(stake, dk[1])
    a_ev = a_profit * fair[0] - stake * fair[1]
    b_ev = b_profit * fair[1] - stake * fair[0]

    dkfair = fair_odds(dk[0], dk[1])

    if a_ev > b_ev:
        bet = bankroll * kelly_criterion(1, dkfair[0], dkfair[1])
        width_a = width(pin[0], pin[1])
        return 0, a_ev, dk[0], bet, width_a
    else:
        bet = bankroll * kelly_criterion(1, dkfair[0], dkfair[1])
        width_b = width(pin[0], pin[1])
        return 1, b_ev, dk[1], bet, width_b

SPORTBOOK = 'BetMGM'
def main(api_key, regions, odds_format, markets, stake, bankroll):
    """
    Main function to fetch, process, and analyze sports odds data.
    """
    data = spreads(api_key, regions, odds_format, markets)
    
    pivoted_data = pivot_and_clean_data(data)
    
    test = pivoted_data[['date', 'sport', 'home_team', 'away_team', 'spread', SPORTBOOK, 'BetOnline.ag', 'Bovada']]
    test['check'] = np.where((test[SPORTBOOK] == 'N/A') | (test['Bovada'] == 'N/A') | (test['BetOnline.ag'] == 'N/A'), 1, 0)
    test = test[test['check'] == 0]

    cols = [SPORTBOOK, 'BetOnline.ag', 'Bovada']
    for col in cols:
        test[col] = test[col].astype('int')
    test['team_index'], test['ev'], test['odds'], test['bet'], test['width'] = zip(*test.apply(lambda row: calculate_ev(
        (row[SPORTBOOK], row['BetOnline.ag']),
        (row['BetOnline.ag'], row['Bovada']),
        (row[SPORTBOOK], row['Bovada']),
        stake,
        bankroll
    ), axis=1))

    ## its flipped for some reason
    test['team'] = np.where(test['team_index'] == 1, test['home_team'], test['away_team'])

    return test[['date', 'sport', 'home_team', 'away_team', 'spread', SPORTBOOK, 'BetOnline.ag', 'Bovada', 'team_index', 'ev', 'odds', 'bet', 'width', 'team']]

result_df = main(api_key='287f26ecf5b58ab73781cfaf78310c08', regions='us', odds_format='american', markets='spreads', stake=100, bankroll=1000).sort_values("ev", ascending=False)

result_df.head(10)

bookmaker,date,sport,home_team,away_team,spread,BetMGM,BetOnline.ag,Bovada,team_index,ev,odds,bet,width,team
1,2023-10-09T22:07:00Z,baseball_mlb,Atlanta Braves,Philadelphia Phillies,-1.5,135,135,135,1,17.500000,135,0.000000,0,Atlanta Braves
19,2023-10-10T01:07:00Z,baseball_mlb,Los Angeles Dodgers,Arizona Diamondbacks,-1.5,125,130,130,1,14.368132,130,10.989011,0,Los Angeles Dodgers
18,2023-10-10T01:07:00Z,baseball_mlb,Los Angeles Dodgers,Arizona Diamondbacks,1.5,-155,-149,-150,1,-16.658239,-149,7.833921,99,Los Angeles Dodgers
0,2023-10-09T22:07:00Z,baseball_mlb,Atlanta Braves,Philadelphia Phillies,1.5,-160,-156,-160,1,-17.747415,-156,4.906771,116,Atlanta Braves
